In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sys
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [3]:
# Funciones

# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
def EncoderViewMinable(df):
    new_dataset = df
    norm = MinMaxScaler()
    norm = norm.fit(new_dataset.values[:,:])
    valMin = norm.data_min_
    valMax = norm.data_max_
    dataRange = norm.data_range_
    #df_norm = norm.fit_transform(df.values[:,:-1])

    return [valMin, valMax, dataRange]


'''
Funcion para generar el vector de pesos aleatorio.
Entradas:
w: Vector de pesos w [0,1], igual al numero de caracteristicas normalizadas.
nc: Nunero de ceros que debe contener el vector de pesos [10,20,30,40,50]
'''
def GenerateWeightVector(w, nc):
    posceros = np.random.choice(len(w), nc, replace=False)
    #print("Posiciones ceros: ", posceros)
    w[posceros] = 0
    w = np.round(w,3)
    s  = np.sum(w)
    wf = np.round(w/s, 3)
    sc = abs(1- np.sum(wf))
    #print("resto pesos: ",sc)
    pos = np.random.randint(len(wf))
    if sc != 0:
        wf[pos]= wf[pos]+sc

    return wf





'''
-Función de calidad, que retorna la metrica de calidad asociada a ese vector w especifico
-Se debe tener en cunata la seleccion de la metrica de calidad asociada, para evaluar
el desempeño del algoritmo (Problema de Clasificacion)
Entradas:
    df_norm: Dataset Normalizado.
    wi: Vector de Pesos.
'''

# Dependiendo del vector de pesos me extrae el acuracy - F1 score
def qualityFunction(df_norm, wi,df):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    minDep = sys.float_info.max
    posMinDep = 0
    for i in range(len(df_norm)):
        vrf = df_norm[i] 
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
        #print(f"vector {i} es muy similar el vector en la posición {posMinDep}, con distancia de {minDep}")
        y_pred.append(df.values[posMinDep][-1])
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return qs




'''
Función para generar la memoria Armonica
Entradas:
    MAC: Tamaño de la  Memoria Armonica
    wi: Vector de Pesos.
    nc: Numero de ceros (Selección de atributos)
'''

def GenerateArmonyMemory(df_norm, MAC,nc,df):
    Lw = []
    #np.random.seed(123)
    for i in range (MAC):
        # Creo la semilla
        
        vp = np.random.rand(df_norm.shape[1])
        #print("Vector de pesos Original: ", vp)
        wi = GenerateWeightVector(vp, nc)
        #print("Vector de pesos Normalizado: ", wi)
        Qs = qualityFunction(df_norm, wi,df)
        #print("Qs: ",Qs )
        wiq = np.append(wi, Qs)
        Lw.append(wiq)

    
    Lw.sort(key=lambda x: x[-1], reverse=True)
    return Lw



def ImprovisationGBHS(PAR, hmn, nc,dataset_normalizado , lmp, HMRC,df):

    df_norm = dataset_normalizado
    np.random.seed(123)
    # Se genera la memoria Armonica - diferentes tamaños
    MA = GenerateArmonyMemory(df_norm,hmn,nc,df)
    #print("MA: ", MA)
    P=len(MA[0]) 

    curvaFitnes = []
    vectorIteration= []
    for i in range (lmp):
        pesosAleatorios = np.random.rand(P-1)
        for j in range(P-1):
            Aleatorio1 = random.random()
            #print("Aleatorio 1: ", Aleatorio1) 
            if (Aleatorio1 < HMRC):
                pma = random.randint(0, hmn-1)
                pesosAleatorios[j]= MA[pma][j]

                Aleatorio2 = random.random()
                if Aleatorio2 < PAR:
                    pesosAleatorios[j] = MA[0][j]
            
            else:
                Aleatorio3 = random.random()
                if Aleatorio3 < nc/P:
                    Aleatorio4 = 0
                else:
                    Aleatorio4 = Aleatorio3/(P-nc)
                
                pesosAleatorios[j] = Aleatorio4
        

        # Normalización de los pesos
        wf = GenerateWeightVector(pesosAleatorios, 0)
        #print("Nuevo vector analizar el fitnes: ", wf)
        fitnes = qualityFunction(df_norm, wf,df)
        #print("Nuevo Fitnes: ", fitnes)



        # Remplazo
        if MA[hmn -1][P-1] < fitnes:
            new_register = np.append(wf, fitnes)
            MA[hmn-1] = new_register
            #print("------------------------------------")
            MA.sort(key=lambda x: x[-1], reverse=True)
        

        
        curvaFitnes.append(MA[0][P-1])
        vectorIteration.append(MA[0])
        

    
    '''
    print("Valor de la curva en la ultima poisción: ",curvaFitnes[-1])

    dicc = {"vector":vectorIteration,
                "Fitnes": curvaFitnes}
    

    df_new = pd.DataFrame(data=dicc)
    df_new.to_csv(f"ResultadosImprovisacion/GBHS.csv")
    dicc = dict()
    '''
    return [curvaFitnes, vectorIteration[-1]]


# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado




In [4]:
#  CARGAR EL DATASEET PRINCIPAL CON LOS  RESPECTIVOS GRUPOS
# ==============================================================================
df = pd.read_csv('Iris.csv')
df_groups = df.copy()
df1 = df.copy()
df1 = df1.drop(["species"], axis= 1)


In [5]:
df_groups['Grupo'] = df_groups['species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
df_groups = df_groups.drop(["species"], axis= 1)
#dataset_train, dataset_test = train_test_split(df_groups, test_size = 0.05, random_state=43)
#print("Longitud Dataset Entrenamiento:",  dataset_train.shape)

In [6]:
valMin, valMax, dataRange = EncoderViewMinable(df1)
print("Longitudes : ", len(valMin), len(valMax), len(dataRange))
# Guardamos los encoders apra posteriores usos
np.savetxt('Encoder_ValMin.txt', valMin)
np.savetxt('Encoder_dataRange.txt', dataRange)



Longitudes :  4 4 4


In [10]:
# Definición de Pesos
# =================================================================================================

#1.Union grupos | unico dataset
#final_datset_join = dataset_train.reset_index(drop=True)
final_dataset_join= df_groups
#2.Normalizacion dataset

#df_norm = NormalizeViewMinable(final_datset_join.values[:,:-1],valMin, dataRange)
#print(df_norm.shape)


In [11]:
final_dataset_join

,sepal_length,sepal_width,petal_length,petal_width,Grupo
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [17]:
#print("Valmin: ", valMin)
#print("DataRange: ", dataRange)

In [18]:
#df_norm[0:5]

In [12]:
lmp = 10                                                                                
HMRC = 0.85                                          
PAR_array=[0.25,0.3, 0.35,0.40]                                                         
hmns_array = [5,10,15,20]  
nc_array = [0,1,2]                                        


#1. Se contruyen los grupos de calidad
#df =final_datset_join
# Lectura de los Encoders

df_norm = NormalizeViewMinable(final_dataset_join.values[:,:-1],valMin, dataRange)


cont = 1

# 
# GBHS - IMPROVISACIÓN
# ============================================================================================
for incero in range(len(nc_array)):
    for ihmn in range(len(hmns_array)):
        for ipar in range(len(PAR_array)):
            print(f"PAR: {PAR_array[ipar]}, Longitud de la Memoria Armonica: {hmns_array[ihmn]}, Numero de ceros: {nc_array[incero]} ")
            print(f"Iteracion {cont}")
            # Parametros Bucle
            PAR = PAR_array[ipar]
            hmn = hmns_array[ihmn]
            nc= nc_array[incero]

            # Se genera la memoria Armonica - diferentes tamaños
            #np.random.seed(43)
            MA = GenerateArmonyMemory(df_norm,hmn,nc,final_dataset_join)
            P=len(MA[0]) 

            curvaFitnes = []
            vectorIteration= []
            for i in range (lmp):
                pesosAleatorios = np.random.rand(P-1)
                for j in range(P-1):
                    Aleatorio1 = random.random() 
                    if (Aleatorio1 < HMRC):
                        pma = random.randint(0, hmn-1)
                        pesosAleatorios[j]= MA[pma][j]

                        Aleatorio2 = random.random()
                        if Aleatorio2 < PAR:
                            pesosAleatorios[j] = MA[0][j]
                    
                    else:
                        Aleatorio3 = random.random()
                        if Aleatorio3 < nc/P:
                            Aleatorio4 = 0
                        else:
                            Aleatorio4 = Aleatorio3/(P-nc)
                        
                        pesosAleatorios[j] = Aleatorio4
                

                # Normalización de los pesos
                wf = GenerateWeightVector(pesosAleatorios, 0)
                fitnes = qualityFunction(df_norm, wf, final_dataset_join)



                # Remplazo
                if MA[hmn -1][P-1] < fitnes:
                    new_register = np.append(wf, fitnes)
                    MA[hmn-1] = new_register
                    #print("------------------------------------")
                    MA.sort(key=lambda x: x[-1], reverse=True)
                

                
                curvaFitnes.append(MA[0][P-1])
                vectorIteration.append(MA[0])
                
 
            
            print("Valor de la curva en la ultima poisción: ",curvaFitnes[-1])

            dicc = {"vector":vectorIteration,
                        "Fitnes": curvaFitnes}
            
    
            df_new = pd.DataFrame(data=dicc)
            df_new.to_csv(f"Resultados/accuracy_PAR_{PAR}_Tmem_{hmn}_nc_{nc}.csv")
            cont=cont+1
            dicc = dict()

PAR: 0.25, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 1
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 2
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 3
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 5, Numero de ceros: 0 
Iteracion 4
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 5
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 6
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 10, Numero de ceros: 0 
Iteracion 7
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud d

C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 5, Numero de ceros: 2 
Iteracion 35
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 5, Numero de ceros: 2 
Iteracion 36
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 37
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 38


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 39


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 10, Numero de ceros: 2 
Iteracion 40
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 41
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 42
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 43
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 15, Numero de ceros: 2 
Iteracion 44
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.25, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 45


C:\Users\germanm\AppData\Local\Temp\ipykernel_6996\632926467.py:28: RuntimeWarning: invalid value encountered in true_divide
  wf = np.round(w/s, 3)


Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.3, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 46
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.35, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 47
Valor de la curva en la ultima poisción:  0.3333333333333333
PAR: 0.4, Longitud de la Memoria Armonica: 20, Numero de ceros: 2 
Iteracion 48
Valor de la curva en la ultima poisción:  0.3333333333333333


In [55]:
#3.Optimización GBHS
# Se jecuta la mejor metahurtistica de acuerdo a los resultados obtenidos
metric,vector_pesos =  ImprovisationGBHS(0.35, 5, 2, df_norm, 50, 0.85, final_datset_join)
#metric,vector_pesos =  ImprovisationGBHS(0.4, 20, 50, df_norm, 30, 0.85, final_datset_join)
vector_pesos_optmizacion = vector_pesos[0: -1]
print("-------------------------- RESULTADOS OPTMIZACIÓN-------------------------")
print("Mejor Fitnes: ",metric)
print("Longitud del vector de pesos: ", len(vector_pesos_optmizacion))

Qs:  0.3380281690140845
Qs:  0.34507042253521125
Qs:  0.3380281690140845
Qs:  0.34507042253521125
Qs:  0.34507042253521125
-------------------------- RESULTADOS OPTMIZACIÓN-------------------------
Mejor Fitnes:  [0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.34507042253521125, 0.3450704

In [40]:
df_norm.shape

(142, 4)

In [38]:
MA = GenerateArmonyMemory(df_norm,5,1,final_datset_join)

vector 0 es muy similar el vector en la posición 34, con distancia de 0.0009604948689712412
vector 1 es muy similar el vector en la posición 24, con distancia de 0.00046701388888888964
vector 2 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 3 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 4 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 5 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 6 es muy similar el vector en la posición 14, con distancia de 6.118931341568502e-05
vector 7 es muy similar el vector en la posición 119, con distancia de 0.0
vector 8 es muy similar el vector en la posición 119, con distancia de 0.0
vector 9 es muy similar el vector en la posición 119, con distancia de 0.0
vector 10 es muy similar el vector en la posición 119, con distancia de 0.0
vector 11 es muy similar el vector en la posición 119,

vector 43 es muy similar el vector en la posición 13, con distancia de 0.0
vector 44 es muy similar el vector en la posición 13, con distancia de 0.0
vector 45 es muy similar el vector en la posición 13, con distancia de 0.0
vector 46 es muy similar el vector en la posición 13, con distancia de 0.0
vector 47 es muy similar el vector en la posición 13, con distancia de 0.0
vector 48 es muy similar el vector en la posición 13, con distancia de 0.0
vector 49 es muy similar el vector en la posición 13, con distancia de 0.0
vector 50 es muy similar el vector en la posición 13, con distancia de 0.0
vector 51 es muy similar el vector en la posición 13, con distancia de 0.0
vector 52 es muy similar el vector en la posición 13, con distancia de 0.0
vector 53 es muy similar el vector en la posición 13, con distancia de 0.0
vector 54 es muy similar el vector en la posición 13, con distancia de 0.0
vector 55 es muy similar el vector en la posición 13, con distancia de 0.0
vector 56 es muy similar 

In [27]:
MA

[array([0.457     , 0.319     , 0.224     , 0.        , 0.34507042]),
 array([0.279     , 0.        , 0.253     , 0.47      , 0.34507042]),
 array([0.474     , 0.        , 0.198     , 0.328     , 0.34507042]),
 array([0.319     , 0.501     , 0.        , 0.18      , 0.34507042]),
 array([0.        , 0.269     , 0.213     , 0.518     , 0.33802817])]

In [33]:
df_norm

array([[0.33333333, 0.16666667, 0.45762712, 0.375     ],
       [0.16666667, 0.16666667, 0.38983051, 0.375     ],
       [0.02777778, 0.41666667, 0.05084746, 0.04166667],
       [0.69444444, 0.33333333, 0.6440678 , 0.54166667],
       [0.94444444, 0.33333333, 0.96610169, 0.79166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.47222222, 0.375     , 0.59322034, 0.58333333],
       [0.52777778, 0.375     , 0.55932203, 0.5       ],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.41666667, 0.33333333, 0.69491525, 0.95833333],
       [0.55555556, 0.54166667, 0.84745763, 1.        ],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.13888889, 0.41666667, 0.06779661, 0.08333333],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.5       , 0.25      , 0.77966102, 0.54166667],
       [0.52777778, 0.33333333, 0.6440678 , 0.70833333],
       [0.94444444, 0.25      ,